In [1]:
#datasets path

#AD100

TRAIN_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/train.csv"
TEST_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/verification-subject-disjoint.csv"
IMAGE_PATH_AD = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/AD100/images/"

#LG4000

TRAIN_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/train.csv"
TEST_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/verification-subject-disjoint.csv"
IMAGE_PATH_LG = "/content/drive/MyDrive/datasets/ND-Contact-Lens/ND-Contact-Lens/LG4000/images/"

In [2]:
import csv
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
from keras.applications import  ResNet50,InceptionV3
from keras.models import Model
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, LeakyReLU, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam, SGD
from keras.optimizers.schedules import ExponentialDecay
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from keras.activations import swish

In [3]:
#Classifier Functions

def contacts_classifier(label):
  if label == "No":
    return [0, 0, 1]
  elif label == "Yes":
    return [0, 1, 0]
  elif label == "Cosmetic":
    return [1, 0, 0]



def eye_classifier(label):
  if label == "Right":
    return [0, 1]
  elif label == "Left":
    return [1, 0]

In [4]:
#Funcao que le o arquivo csv relacionado aos datasets
def read_csv(filepath):
  lines = []

  with open(filepath, "r") as csv_read:
    csvreader = csv.reader(csv_read)

    for row in csvreader:
      lines.append(np.asarray(row))
    csv_read.close()

    lines.pop(0)
    lines = np.array(lines).T

    return lines

In [5]:
#Funcao de conversao dos labels de classificacao das classes
def convert_classes(filepath):
  csv_data = read_csv(filepath)

  eye, contacts = [], []

  for label1 in csv_data[5]:
    contacts.append(contacts_classifier(label1))

  for label2 in csv_data[2]:
    eye.append(eye_classifier(label2))

  return np.array(eye).astype(np.int32), np.array(contacts).astype(np.int32)

In [6]:
#Conversao das classes em numeros inteiros
y_train_eye, y_train_contact = convert_classes(TRAIN_PATH_AD)
y_test_eye, y_test_contact = convert_classes(TEST_PATH_AD)

print(y_train_eye.shape, y_train_contact.shape)
print(y_test_eye.shape, y_test_contact.shape)

(600, 2) (600, 3)
(300, 2) (300, 3)


In [7]:
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf

#Funcao que le as imagens e transformam elas em arrays numpy
def read_images(filepath, augment=False):
  csv_data = read_csv(filepath)
  images_names = csv_data[0]

  images = []

  for item in images_names:
    img_path = IMAGE_PATH_AD + item + ".tiff"
    image = img_to_array(load_img(img_path, target_size=(224, 224))) / 255.0

    images.append(image)

  return np.array(images).astype(np.float32)

In [8]:
x_train = read_images(TRAIN_PATH_AD)
x_test = read_images(TEST_PATH_AD)

print(x_train.shape)
print(x_test.shape)

(600, 224, 224, 3)
(300, 224, 224, 3)


In [47]:
from keras.applications import Xception

#Modelo base
base_model = InceptionV3(include_top=False, weights='imagenet')
base_model.trainable = True

base_model.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, None, None, 3)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_470 (Conv2D)       │ (None, None, None, 32) │            864 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_475   │ (None, None, None, 32) │             96 │ conv2d_470[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_470            │ (None, None, None, 32) │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_471 (Conv2D)       │ (None, None, None, 32) │          9,216 │ activation_470[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_476   │ (None, None, None, 32) │             96 │ conv2d_471[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_471            │ (None, None, None, 32) │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_472 (Conv2D)       │ (None, None, None, 64) │         18,432 │ activation_471[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_477   │ (None, None, None, 64) │            192 │ conv2d_472[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_472            │ (None, None, None, 64) │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_20          │ (None, None, None, 64) │              0 │ activation_472[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_473 (Conv2D)       │ (None, None, None, 80) │          5,120 │ max_pooling2d_20[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_478   │ (None, None, None, 80) │            240 │ conv2d_473[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_473            │ (None, None, None, 80) │              0 │ batch_normalization_4… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_474 (Conv2D)  

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [48]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Flatten
from keras.applications import xception


#Fine tunning do Modelo pre treinado usado
input_layer = Input(shape=(224, 224, 3))

base_model = base_model (input_layer)

batch_norm = BatchNormalization() (base_model)
global_pooling = GlobalAveragePooling2D() (batch_norm)

dropout1 = Dropout(0.35)(global_pooling)
dense1 = Dense(128, activation='relu')(dropout1)

dropout2 = Dropout(0.35)(dense1)
dense2 = Dense(128, activation='relu')(dropout2)

dropout3 = Dropout(0.35)(dense2)
dense3 = Dense(64, activation='relu')(dropout3)

final_dropout = Dropout(0.35)(dense3)
final_dense   = Dense(32, activation='relu')(final_dropout)

#contacts layer
contacts_layer = Dense(3, activation='softmax', name="contacts_layer")(final_dense)

#eye layer
eye_layer = Dense(2, activation='sigmoid', name="eye_layer")(final_dense)

model = Model(inputs= input_layer, outputs=[eye_layer, contacts_layer])

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inception_v3 (Functional) │ (None, 5, 5, 2048)     │     21,802,784 │ input_layer_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_569   │ (None, 5, 5, 2048)     │          8,192 │ inception_v3[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 2048)           │              0 │ batch_normalization_5… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_22 (Dropout)      │ (None, 2048)           │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_22 (Dense)          │ (None, 128)            │        262,272 │ dropout_22[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_23 (Dropout)      │ (None, 128)            │              0 │ dense_22[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_23 (Dense)          │ (None, 128)            │         16,512 │ dropout_23[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_24 (Dropout)      │ (None, 128)            │              0 │ dense_23[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_24 (Dense)          │ (None, 64)             │          8,256 │ dropout_24[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 64)             │              0 │ dense_24[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_25 (Dense)          │ (None, 32)             │          2,080 │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ eye_layer (Dense)         │ (None, 2)              │             66 │ dense_25[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ contacts_layer (Dense)    │ (None, 3)              │             99 │ dense_25[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 22,100,261 (84.31 MB)

 Trainable params: 22,061,733 (84.16 MB)

 Non-trainable params: 38,528 (150.50 KB)

In [49]:
from keras.optimizers import RMSprop, Adam, SGD
from keras.optimizers.schedules import ExponentialDecay
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy


#Definicao de hiperparametros, funcao de otimizacao e funcoes de loss para as tarefas
learning_rate = 0.00001
epochs = 300
batch_size = 16


optimizer = Adam(learning_rate=learning_rate)


losses = {
    'eye_layer': BinaryCrossentropy(),
    'contacts_layer': CategoricalCrossentropy()
}

model.compile(optimizer= optimizer, loss= losses, metrics= ['accuracy', 'accuracy'])

In [50]:
#Treinamento do modelo
history = model.fit(x_train, [y_train_eye, y_train_contact], epochs= epochs, batch_size=batch_size, validation_split=0.2, shuffle=True)

Epoch 1/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 86s 510ms/step - contacts_layer_accuracy: 0.2612 - contacts_layer_loss: 1.1987 - eye_layer_accuracy: 0.5363 - eye_layer_loss: 0.7248 - loss: 1.9235 - val_contacts_layer_accuracy: 0.3333 - val_contacts_layer_loss: 1.0155 - val_eye_layer_accuracy: 0.4750 - val_eye_layer_loss: 0.7503 - val_loss: 1.7664
Epoch 2/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 41s 165ms/step - contacts_layer_accuracy: 0.2707 - contacts_layer_loss: 1.1835 - eye_layer_accuracy: 0.5460 - eye_layer_loss: 0.7424 - loss: 1.9259 - val_contacts_layer_accuracy: 0.3917 - val_contacts_layer_loss: 1.0381 - val_eye_layer_accuracy: 0.4833 - val_eye_layer_loss: 0.7467 - val_loss: 1.7789
Epoch 3/300
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - contacts_layer_accuracy: 0.3294 - contacts_layer_loss: 1.1404 - eye_layer_accuracy: 0.5573 - eye_layer_loss: 0.7169 - loss: 1.8573 - val_contacts_layer_accuracy: 0.4917 - val_contacts_layer_loss: 1.0096 - val_eye_layer_accuracy: 0.5000 - val_eye_layer_loss: 0.7299 

In [51]:
#Teste do modelo
evaluation = model.evaluate(x_test, [y_test_eye, y_test_contact], verbose=1)

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 437ms/step - contacts_layer_accuracy: 0.6529 - contacts_layer_loss: 2.3262 - eye_layer_accuracy: 0.9959 - eye_layer_loss: 0.0207 - loss: 2.3692


In [52]:
#Resultado das predicoes
print("Eye's Prediction:     ",evaluation[4] * 100,"%")
print("Contact's Prediction: ",evaluation[3] * 100,"%")

Eye's Prediction:      98.66666793823242 %
Contact's Prediction:  70.6666648387909 %
